# xpander.ai single agent using local tools for mathematics calculations combined with OpenAI

In [ ]:
from xpander_sdk import XpanderClient, ToolCallResult
from openai import OpenAI
from dotenv import load_dotenv
from os import environ

load_dotenv()


In [2]:
OPENAI_API_KEY = environ["OPENAI_API_KEY"]
XPANDER_API_KEY = environ["XPANDER_API_KEY"]
XPANDER_AGENT_ID = environ["XPANDER_AGENT_ID_MULTI"]

## Initialize xpander.ai Client

In [3]:
xpander_client = XpanderClient(api_key=XPANDER_API_KEY)

## Initialize OpenAI Client

In [4]:
openai_client = OpenAI(api_key=OPENAI_API_KEY)

## Load agent

In [ ]:
agent = xpander_client.agents.get(agent_id=XPANDER_AGENT_ID)

## Invoke the agent with query and create an execution task

In [ ]:
agent.add_task("""
Search for 2 startups in the AI sector and get LinkedIn profiles of their founders.
""")

## Init the agent's memory with input (query) and instructions

In [7]:
agent.memory.init_messages(input=agent.execution.input_message,instructions=agent.instructions)

## Run the agent and wait for result

In [8]:
while not agent.is_finished():
    response = openai_client.chat.completions.create(
                model= "gpt-4o",
                messages=agent.messages,
                tools=agent.get_tools(),
                tool_choice=agent.tool_choice,
                temperature=0.0
        )
            
    # add messages directly from the LLM response
    agent.add_messages(response.model_dump())
    
    # extract tool calls from the LLM response
    tool_calls = XpanderClient.extract_tool_calls(llm_response=response.model_dump())
    # run tools
    agent.run_tools(tool_calls=tool_calls)


## Fetch & print the agent execution result

In [ ]:
execution_result = agent.retrieve_execution_result()

print("status", execution_result.status)
print("result", execution_result.result)